In [2]:
from src.config import *
import pandas as pd
import numpy as np 

In [3]:
df = pd.read_csv('Dataset/transformed/transformed_data_6h.csv')

In [7]:
imp_lags = ['rides_t-1', 'rides_t-4', 'rides_t-28', 'rides_t-24', 'rides_t-3', 'rides_t-84', 'rides_t-56', 'rides_t-5', 'rides_t-52', 'rides_t-168','pickup_hour', 'target']

df_2 = df[imp_lags]

In [8]:
df_2

,rides_t-1,rides_t-4,rides_t-28,rides_t-24,rides_t-3,rides_t-84,rides_t-56,rides_t-5,rides_t-52,rides_t-168,pickup_hour,target
0,15,6,1,0,6,7,7,16,2,16,2022-01-01 00:00:00,8
1,8,6,0,4,22,6,0,6,9,14,2022-01-01 06:00:00,4
2,4,22,3,15,15,29,11,6,22,52,2022-01-01 12:00:00,4
3,4,15,3,11,8,24,12,22,14,16,2022-01-01 18:00:00,3
4,3,8,0,3,4,10,2,15,2,12,2022-01-02 00:00:00,2
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12,5,2,3,3,12,7,10,0,11,2023-12-30 18:00:00,4
8756,4,3,0,0,4,2,0,5,1,0,2023-12-31 00:00:00,2
8757,2,4,2,4,12,7,4,3,3,7,2023-12-31 06:00:00,6
8758,6,12,7,6,4,10,13,4,9,6,2023-12-31 12:00:00,6


In [10]:
# Ensure the 'pickup_hour' column is in datetime format
df_2['pickup_hour'] = pd.to_datetime(df_2['pickup_hour'])

# Split the data into training (2022) and testing (2023)
train_data = df_2[df_2['pickup_hour'].dt.year == 2022]
test_data = df_2[df_2['pickup_hour'].dt.year == 2023]

# Separate features (X) and target (Y)
X_train = train_data.drop(columns=['target','pickup_hour'])
y_train = train_data['target']

X_test = test_data.drop(columns=['target','pickup_hour'])
y_test = test_data['target']

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4380, 10)
(4380,)
(4380, 10)
(4380,)


C:\Users\chanc\AppData\Local\Temp\ipykernel_23028\343135844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['pickup_hour'] = pd.to_datetime(df_2['pickup_hour'])


In [11]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# 2. Define and train LightGBM model
model = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=100,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

model.fit(X_train, y_train)

# 3. Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"LightGBM MAE: {mae:.2f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000278 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 730
[LightGBM] [Info] Number of data points in the train set: 4380, number of used features: 10
[LightGBM] [Info] Start training from score 12.978082
LightGBM MAE: 4.80


In [13]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import dagshub
dagshub.init(repo_owner='C-bundele', repo_name='City_Bike', mlflow=True)

from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow

mlflow = set_mlflow_tracking()

import mlflow

print("Current MLflow Tracking URI:", mlflow.get_tracking_uri())

mlflow.set_experiment("Lightgbm with feature importance Model")

# Start an MLflow run
with mlflow.start_run():
        mlflow.log_metric("mean_absolute_error",mae)

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/C-bundele/City_Bike "HTTP/1.1 200 OK"


Initialized MLflow to track repo "C-bundele/City_Bike"

INFO:dagshub:Initialized MLflow to track repo "C-bundele/City_Bike"


Repository C-bundele/City_Bike initialized!

INFO:dagshub:Repository C-bundele/City_Bike initialized!


Current MLflow Tracking URI: https://dagshub.com/C-bundele/City_Bike.mlflow
🏃 View run secretive-turtle-563 at: https://dagshub.com/C-bundele/City_Bike.mlflow/#/experiments/2/runs/e5d5ee3a81a34cbcb1867e7241e241f4
🧪 View experiment at: https://dagshub.com/C-bundele/City_Bike.mlflow/#/experiments/2
